In [6]:
import math
import pandas as pd

# Load the CSV file
file_path = "Steel_Sections.csv"
sections_df = pd.read_csv(file_path)

# Convert the DataFrame to a dictionary for structural analysis
sections_data = sections_df.set_index("Section").T.to_dict()

def check_local_stability(section_type, b_f, t_f, h_w, t_w, d, Fy, E):
    try:
        if section_type == "W-section":
            bt_flange_ratio = b_f / (2 * t_f)
            flange_limit = 0.56 * math.sqrt(E / Fy)
            hw_tw_ratio = h_w / t_w
            web_limit = 3.76 * math.sqrt(E / Fy)
            return bt_flange_ratio <= flange_limit and hw_tw_ratio <= web_limit

        elif section_type == "HSS Square":
            bt_ratio = (b_f - 2 * t_f) / t_f
            limit = 1.4 * math.sqrt(E / Fy)
            return bt_ratio <= limit

        elif section_type == "HSS Circular":
            dt_ratio = d / t_f
            limit = 0.11 * E / Fy
            return dt_ratio <= limit

        return True

    except Exception as e:
        print(f"Error in local stability check: {e}")
        return False

def analyze_and_select_column(Fx, L, K, Fy_steel=345, E=200000):
    phi_c = 0.9
    results = []

    for section_name, properties in sections_data.items():
        try:
            d = properties.get("d")
            b_f = properties.get("b", d)
            t_f = properties.get("tf", 0)
            t_w = properties.get("tw", 0)
            h_w = d - 2 * t_f if t_f else 0
            A = properties.get("A", 0)
            Ix = properties.get("Ix", 0) * 1e6  # Convert Ix to mm^4
            Iy = properties.get("Iy", Ix) * 1e6  # Convert Iy to mm^4
            J = properties.get("J", 0) * 1e3  # Convert J to mm^4 from x10^-3
            section_type = properties.get("Type")

            if not all([d, b_f, t_f, t_w, A, Ix]):
                continue

            if not check_local_stability(section_type, b_f, t_f, h_w, t_w, d, Fy_steel, E):
                continue

            Le = K * L * 1000
            r_x = math.sqrt(Ix / A)
            r_y = math.sqrt(Iy / A)
            Fe_x = (math.pi**2 * E) / (Le / r_x)**2
            Fe_y = (math.pi**2 * E) / (Le / r_y)**2
            Fe_flexural = min(Fe_x, Fe_y)

            Fe_torsional = (
                (math.pi**2 * E) / (Le / math.sqrt(J / A))**2 if J > 0 else Fe_flexural
            )
            Fe = min(Fe_flexural, Fe_torsional)

            slenderness_ratio = Le / r_x
            if slenderness_ratio > 200:
                continue

            Pn = Fe * A / 1e6
            Pu = phi_c * Pn

            demand_capacity_ratio = Fx / Pu if Pu > 0 else float("inf")

            # Change here to filter sections with demand-capacity ratio <= 0.99
            if demand_capacity_ratio > 0.99:
                continue

            results.append({
                "Section": section_name,
                "Type": section_type,
                "A (mm²)": A,
                "Pu (kN)": round(Pu, 2),
                "Pn (kN)": round(Pn, 2),
                "Fe (MPa)": round(Fe, 2),
                "Slenderness": round(slenderness_ratio, 2),
                "Demand/Capacity Ratio": round(demand_capacity_ratio, 2),
            })

        except Exception as e:
            print(f"Error analyzing section {section_name}: {e}")

    results_df = pd.DataFrame(results)
    if not results_df.empty:
        results_df["Optimization"] = (results_df["Demand/Capacity Ratio"] - 0.99).abs()
        results_df = results_df.sort_values(by=["Optimization", "A (mm²)"], ascending=[True, True])
        return results_df.iloc[0]
    return None

Fx = 900  # Axial load in kN
L = 3  # Length in meters
K = 0.5 # Effective length factor

economical_column = analyze_and_select_column(Fx, L, K)

if economical_column is not None:
    print(f"Most economical column: {economical_column['Section']}")
    print(economical_column)
else:
    print("No suitable column found.")


Most economical column: W40X655
Section                    W40X655
Type                     W-section
A (mm²)                     125000
Pu (kN)                     939.59
Pn (kN)                    1043.98
Fe (MPa)                   8351.88
Slenderness                   3.46
Demand/Capacity Ratio         0.96
Optimization                  0.03
Name: 0, dtype: object
